<a href="https://colab.research.google.com/github/praveenkumarbalakrishnan/tasks/blob/main/diabetes_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from azureml.core import Workspace

ws = Workspace.create(name='diabetes_prediction',
                    subscription_id='84bd4663-5df3-49e8-943e-ba5879616bac',
                    resource_group='house_prediction_group',
                    create_resource_group=True,
                    location='Central India'
                    )

get_workspace error using subscription_id=84bd4663-5df3-49e8-943e-ba5879616bac, resource_group_name=house_prediction_group, workspace_name=diabetes_prediction


Deploying AppInsights with name diabetesinsights38b16d07.
Deployed AppInsights with name diabetesinsights38b16d07. Took 6.69 seconds.
Deploying StorageAccount with name diabetesstoragea42024b6c.
Deploying KeyVault with name diabeteskeyvault46429964.
Deployed KeyVault with name diabeteskeyvault46429964. Took 19.36 seconds.
Deploying Workspace with name diabetes_prediction.
Deployed StorageAccount with name diabetesstoragea42024b6c. Took 30.53 seconds.
Deployed Workspace with name diabetes_prediction. Took 50.92 seconds.


In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core import ComputeTarget

provisioning_config = AmlCompute.provisioning_configuration(
    vm_size="STANDARD_D1_V2",
    min_nodes=1,
    max_nodes=4
)
aml_compute = ComputeTarget.create(ws, "diabetes-prediction-compute", provisioning_config)
aml_compute.wait_for_completion(show_output=True)

InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [ ]:
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment(name="diabetes_prediction_env")
conda_dep = CondaDependencies()

# Installs dependent packages
conda_dep.add_pip_package("numpy")
conda_dep.add_pip_package("scikit-learn")
conda_dep.add_pip_package("pandas")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

In [ ]:
import os
from azureml.core.experiment import Experiment
from azureml.core import ScriptRunConfig

script_run_config = ScriptRunConfig(source_directory=os.getcwd(), script="train.py", compute_target=aml_compute, environment=myenv)
experiment = Experiment(workspace=ws, name="diabeted-prediction-experiment")
run = experiment.submit(config=script_run_config)
run.display_name = 'run-4'
run.wait_for_completion(show_output=True)

RunId: diabeted-prediction-experiment_1680188236_1df257eb
Web View: https://ml.azure.com/runs/diabeted-prediction-experiment_1680188236_1df257eb?wsid=/subscriptions/84bd4663-5df3-49e8-943e-ba5879616bac/resourcegroups/house_prediction_group/workspaces/diabetes_prediction&tid=d02378ec-1688-46d5-8540-1c28b5f470f6

Streaming azureml-logs/20_image_build_log.txt

2023/03/30 14:57:36 Downloading source code...
2023/03/30 14:57:37 Finished downloading source code
2023/03/30 14:57:37 Creating Docker network: acb_default_network, driver: 'bridge'
2023/03/30 14:57:38 Successfully set up Docker network: acb_default_network
2023/03/30 14:57:38 Setting up Docker configuration...
2023/03/30 14:57:38 Successfully set up Docker configuration
2023/03/30 14:57:38 Logging in to registry: 6ff7b4d0f1bc4373b3c11b440ee621c4.azurecr.io
2023/03/30 14:57:39 Successfully logged into 6ff7b4d0f1bc4373b3c11b440ee621c4.azurecr.io
2023/03/30 14:57:39 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory

{'runId': 'diabeted-prediction-experiment_1680188236_1df257eb',
 'target': 'diabetes-prediction-compute',
 'status': 'Completed',
 'startTimeUtc': '2023-03-30T15:07:57.433761Z',
 'endTimeUtc': '2023-03-30T15:09:39.648305Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlctrain',
  'ContentSnapshotId': '78749803-2583-4ea3-8694-a20485c90969',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'diabetes-prediction-compute',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'credentialPassthrough': False,
  'identity': 

In [ ]:
from azureml.core import Model

model = Model.register(ws, model_path='./best_model.pkl', model_name='daibetes_decision_tree_v2')

Registering model daibetes_decision_tree_v2


In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

inference_config = InferenceConfig(
    source_directory='./', entry_script='./score.py', environment=myenv)

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Uses the specific FPGA enabled VM (sku: Standard_PB6s)
# Standard_PB6s are available in: eastus, westus2, westeurope, southeastasia
prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_A2_v2",
                                                    cluster_purpose="DevTest",
                                                    location = "centralindia")

aks_name = 'scorecompute'
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.model import Model

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 0.1, memory_gb = 0.5)
service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config, aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

/tmp/ipykernel_9347/2978841988.py:8: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, "myservice", [model], inference_config, deployment_config, aks_target)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-03-30 15:47:13+00:00 Creating Container Registry if not exists.
2023-03-30 15:47:13+00:00 Registering the environment.
2023-03-30 15:47:16+00:00 Use the existing image..
2023-03-30 15:47:17+00:00 Creating resources in AKS.
2023-03-30 15:47:18+00:00 Submitting deployment to compute.
2023-03-30 15:47:18+00:00 Checking the status of deployment myservice..
2023-03-30 15:49:58+00:00 Checking the status of inference endpoint myservice.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy
/bin/bash: /azureml-envs/azureml_3b8f062db50884e871427e436b1d8e64/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_3b8f062db50884e871427e436b1d8e64/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bas

In [ ]:
import urllib.request
import json
import os
import ssl

import pandas as pd

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script

data = {
    "data": [
      [7.0, 181.0, 84.0, 21.0, 192.0, 35.9, 0.586, 51.0],
      [2.0, 127.0, 58.0, 24.0, 275.0, 27.7, 1.6, 25.0],
      [11.0, 138.0, 76.0, 29.0, 125.0, 33.2, 0.42, 35.0]]
  }

body = str.encode(json.dumps(data))

url = 'http://20.219.228.250:80/api/v1/service/myservice/score'
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = 'QJ2R7qF0AfWZ9H4kS4gSOQsbpx7aj8Fl'
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")


headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))


b'"{\\"result\\": [1.0, 0.0, 1.0]}"'
